In [1]:
import pandas as pd
import numpy as pd
from bayes_opt import BayesianOptimization
from bayes_opt.logger import JSONLogger
from bayes_opt.event import Events
import lightgbm as lgb
from lightgbm import LGBMClassifier
import numpy as np
from sklearn.model_selection import KFold, train_test_split, cross_val_score
from sklearn.metrics import make_scorer
from utils import g_score

In [3]:
def black_box_function(leaf_size_log, coverage, learning_rate, feature_fraction):
    full_params = {**params_fijos,'learning_rate':learning_rate, 'feature_fraction':feature_fraction}
    full_params['min_data_in_leaf'] = np.maximum(1, np.around(X_train.shape[0] / (2.0 ** leaf_size_log)).astype(int))
    full_params['num_leaves']  =  np.minimum(131072, np.maximum(2, np.around(coverage * X_train.shape[0] / full_params['min_data_in_leaf']).astype(int)))
    #que carajo es 131072
    #del params['leaf_size_log']
    #del params['coverage']
    #full_params = {**params,**params_fijos}
    model = lgb.LGBMClassifier(**full_params)
    return cross_val_score(model, X_train, y_train, scoring=g_score, cv=cv, n_jobs=-1).mean()
    #acá se pierde la varianza de los scores pero bueno...

In [7]:
X_train = np.load('X_train.npy',allow_pickle=True)
y_train = np.load('y_train.npy')
feature_names = list(np.load('feature_names.npy'))
cv = KFold() #ver si hay que usar shuffle o no (sólo ahí random_state.) n_splits = default 5

In [15]:

params_fijos = {
   'boosting': "gbdt",               #puede ir  dart  , ni pruebe random_forest
   'objective': "binary",
   'metric': "custom",
   'first_metric_only': True,
   'boost_from_average': True,
   'feature_pre_filter': False,
   'force_row_wise': True,           #para que los alumnos no se atemoricen con tantos warning
   'verbosity': -100,
   'max_depth':  -1,                 # -1 significa no limitar,  por ahora lo dejo fijo
   'min_gain_to_split': 0.0,         #por ahora, lo dejo fijo
   'min_sum_hessian_in_leaf': 0.001, #por ahora, lo dejo fijo
   'lambda_l1': 0.0,                 #por ahora, lo dejo fijo
   'lambda_l2': 0.0,                 #por ahora, lo dejo fijo
   'max_bin': 31,                    #por ahora, lo dejo fijo
   'num_iterations': 9999,           #un numero muy grande, lo limita early_stopping_rounds

   'bagging_fraction': 1.0,          #por ahora, lo dejo fijo
   'pos_bagging_fraction': 1.0,      #por ahora, lo dejo fijo
   'neg_bagging_fraction': 1.0,      #por ahora, lo dejo fijo

   'drop_rate':  0.1,                #solo se activa en  dart
   'max_drop': 50,                   #solo se activa en  dart
   'skip_drop': 0.5,                 #solo se activa en  dart

   'extra_trees': False,

   'seed': 1
}

pbounds = { 
         "learning_rate":       (0.01,  0.3),
         "feature_fraction":    (0.2,   0.8),
         "coverage":            (0.05,  1.0),
         "leaf_size_log":       (1.0,   12.0)
}

optimizer = BayesianOptimization(
    f=black_box_function,
    pbounds=pbounds,
    random_state=1,
)

# optimizer.probe(
#     params={"x": 0.5, "y": 0.7},
#     lazy=True,
# )

logger = JSONLogger(path="./logs.json")
optimizer.subscribe(Events.OPTIMIZATION_STEP, logger)

In [16]:
optimizer.maximize(
    init_points=2,
    n_iter=10,
)

[LightGBM] [Warning] feature_fraction is set=0.6321946960652949, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6321946960652949
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] min_gain_to_split is set=0.0, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0.0
[LightGBM] [Warning] min_data_in_leaf is set=41963, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=41963
[LightGBM] [Warning] lambda_l2 is set=0.0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.0
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=0.001, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=0.001
[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fraction=1.0
[LightGBM] [Warning] lambda_l1 is set=0.0, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0


/home/user/.local/share/virtualenvs/dmeyf-kP6Zbi_a/lib/python3.8/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/home/user/.local/share/virtualenvs/dmeyf-kP6Zbi_a/lib/python3.8/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] feature_fraction is set=0.6321946960652949, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6321946960652949
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] min_gain_to_split is set=0.0, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0.0
[LightGBM] [Warning] min_data_in_leaf is set=41963, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=41963
[LightGBM] [Warning] lambda_l2 is set=0.0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.0
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=0.001, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=0.001
[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fraction=1.0
[LightGBM] [Warning] lambda_l1 is set=0.0, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0


/home/user/.local/share/virtualenvs/dmeyf-kP6Zbi_a/lib/python3.8/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] feature_fraction is set=0.6321946960652949, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6321946960652949
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] min_gain_to_split is set=0.0, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0.0
[LightGBM] [Warning] min_data_in_leaf is set=41963, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=41963
[LightGBM] [Warning] lambda_l2 is set=0.0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.0
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=0.001, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=0.001
[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fraction=1.0
[LightGBM] [Warning] lambda_l1 is set=0.0, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0


...

/home/user/.local/share/virtualenvs/dmeyf-kP6Zbi_a/lib/python3.8/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/home/user/.local/share/virtualenvs/dmeyf-kP6Zbi_a/lib/python3.8/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] feature_fraction is set=0.44808242476044025, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.44808242476044025
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] min_gain_to_split is set=0.0, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0.0
[LightGBM] [Warning] min_data_in_leaf is set=645, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=645
[LightGBM] [Warning] lambda_l2 is set=0.0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.0
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=0.001, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=0.001
[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fraction=1.0
[LightGBM] [Warning] lambda_l1 is set=0.0, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0


/home/user/.local/share/virtualenvs/dmeyf-kP6Zbi_a/lib/python3.8/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] feature_fraction is set=0.44808242476044025, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.44808242476044025
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] min_gain_to_split is set=0.0, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0.0
[LightGBM] [Warning] min_data_in_leaf is set=645, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=645
[LightGBM] [Warning] lambda_l2 is set=0.0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.0
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=0.001, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=0.001
[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fraction=1.0
[LightGBM] [Warning] lambda_l1 is set=0.0, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0


KeyboardInterrupt: 

In [ ]:
# new_optimizer = BayesianOptimization(
#     f=black_box_function,
#     pbounds={"x": (-2, 2), "y": (-2, 2)},
#     verbose=2,
#     random_state=7,
# )
# load_logs(optimizer, logs=["./logs.json"]);


In [17]:
optimizer.res

[{'target': 0.0,
  'params': {'coverage': 0.4461709044674453,
   'feature_fraction': 0.6321946960652949,
   'leaf_size_log': 1.0012581229907938,
   'learning_rate': 0.09767644606323352}},
 {'target': 32773200.0,
  'params': {'coverage': 0.18941809627625739,
   'feature_fraction': 0.2554031568612787,
   'leaf_size_log': 3.04886232515438,
   'learning_rate': 0.11021261084248384}},
 {'target': 41316400.0,
  'params': {'coverage': 0.05733386318491501,
   'feature_fraction': 0.29711656052761776,
   'leaf_size_log': 8.803059375869982,
   'learning_rate': 0.23177368910756427}}]